In [29]:
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

У меня короче вид задачи Lu = -λpx, этот самый p - меняется. Он либо 1(на 1 части отрезка) либо 4(на второй части отрезка).

То есть задача Lu = -λx или Lu = -4λx. То есть на второй части оператор надо делить на 4. Или смириться с тем, что я ищу не λ, а 4λ.

И снизу Вы м.б. увидите это, если вникать будете, что я нижнюю часть матрицы оператора делю на 4.

Это все мои замечания :0

In [30]:

h = 0.01

e = 1e-14

l = 10.0

x = np.linspace(0, l, int(l/h))
n = len(x)

p = 1 + 3 * np.heaviside(2*x - l, 0)


def create_diagonals_L(n, h):

    main_diag = np.ones(n) * 2
    main_diag[0], main_diag[-1] = 3, 3
    
    upper_diag = -1 * np.ones(n-1)
    lower_diag = -1 * np.ones(n-1)
    
    main_diag /= h**2
    upper_diag /= h**2
    lower_diag /= h**2

    
    # У меня справа от оператора стоит 1 + 3 * np.heaviside(2*x - l, 0), то есть оператора (матрицу) надо поделить на 1 или на 4 соответственно:
    
    for i in range(n):
        main_diag[i] /= p[i]
            
    for i in range(n-1):
        lower_diag[i] /= p[i+1]
        upper_diag[i] /= p[i]

    
    return main_diag, upper_diag, lower_diag


main_diag, upper_diag, lower_diag = create_diagonals_L(n, h)



In [31]:
#Это я так для себя выводил, смотрел что получается
matrix_test = np.zeros((n, n))

np.fill_diagonal(matrix_test, main_diag)

for i in range(n-1):
    matrix_test[i, i+1] = upper_diag[i]

for i in range(1, n):
    matrix_test[i, i-1] = lower_diag[i-1]
print(matrix_test)

[[ 30000. -10000.      0. ...      0.      0.      0.]
 [-10000.  20000. -10000. ...      0.      0.      0.]
 [     0. -10000.  20000. ...      0.      0.      0.]
 ...
 [     0.      0.      0. ...   5000.  -2500.      0.]
 [     0.      0.      0. ...  -2500.   5000.  -2500.]
 [     0.      0.      0. ...      0.  -2500.   7500.]]


In [32]:
#определяем реальные с.з
def lam_k(k_, signum_):
    return (((2*k_-1)*np.pi + signum_ * np.arccos(1/3))/l)**2

def lam_n(k_):
    return (2*np.pi * k_ / l)**2

n_max = 2 #кол-во серий (у меня в серии 3 штучки их)

real_lambds = []
for i in range(1, n_max + 1):
    real_lambds = np.append(real_lambds, lam_k(i, -1))
    real_lambds = np.append(real_lambds, lam_k(i, 1))
    real_lambds = np.append(real_lambds, lam_n(i))

print(real_lambds)

[0.03650519 0.19119212 0.39478418 0.67138662 1.13544739 1.5791367 ]


In [33]:
#Прогонка

def A_k(x, A_k_, main_diag, upper_diag, lower_diag):
    for i in range(1, len(x) - 1):
        A_k_.append(- upper_diag[i] / (main_diag[i] + lower_diag[i - 1] * A_k_[i - 1]))
    A_k_.append(0)


def B_k(x, B_k_, A_k_, main_diag, upper_diag, lower_diag, X):
    for i in range(1, len(x) - 1):
        B_k_.append((X[i] - lower_diag[i - 1] * B_k_[i - 1]) / (main_diag[i] + lower_diag[i - 1] * A_k_[i - 1]))
    B_k_.append((X[-1] - lower_diag[-1] * B_k_[len(x) - 2]) / (main_diag[-1] + lower_diag[-1] * A_k_[len(x) - 2]))


def TMA(main_diag, upper_diag, lower_diag, X, x):
    c_0 = upper_diag[0]
    b_0 = main_diag[0]
    f_0 = X[0]

    A_k_ = [-c_0 / b_0]
    A_k(x, A_k_, main_diag, upper_diag, lower_diag)

    B_k_ = [f_0 / b_0]
    B_k(x, B_k_, A_k_, main_diag, upper_diag, lower_diag, X)

    X_new = np.zeros(len(x))
    X_new[-1] = B_k_[len(x) - 1]
    for i in range(len(x) - 2, -1, -1):
        X_new[i] = B_k_[i] + A_k_[i] * X_new[i + 1]

    return X_new

In [34]:
#Умножение 3-х дм на х

def TMmulpt(main_diag, upper_diag, lower_diag, X):
    result = np.zeros(len(main_diag))

    result[0] = main_diag[0] * X[0] + upper_diag[0] * X[1]
    for i in range(1, len(main_diag) - 1):
        result[i] = lower_diag[i - 1] * X[i - 1] + main_diag[i] * X[i] + upper_diag[i] * X[i + 1]
    result[len(main_diag) - 1] = lower_diag[len(main_diag) - 2] * X[len(main_diag) - 2] + main_diag[len(main_diag) - 1] * X[len(main_diag) - 1]

    return result

In [35]:
#Это для с.ф. надо эту фигню

def B_lam_k(lambda_):
        return 1/(2*np.cos(np.sqrt(lambda_) * l * 0.5))

def B_lam_n(n_):
    return 0.5 * (-1)**(n_ + 1)

def get_B(i_, lambda_):
    if (int(i_) % 3 != 2):
        return B_lam_k(lambda_)
    else:
        return B_lam_n((int)(i_ / 3) + 1)
    

#метод Рэлея

def rayleigh_method(main_diag, upper_diag, lower_diag, init_lambds_, x, count_func = 6, max_iter=1000):
    num_lambds = []
    num_func = []
    init_lambds = init_lambds_
    
    for i in range(count_func):
        
        #задаём нач. с.ф. 
        part1 = np.sin(np.sqrt(init_lambds[i]) * x - e) * np.heaviside((l / 2) - x, 0)
        part2 = get_B(i, init_lambds[i]) * np.sin(2 * np.sqrt(init_lambds[i]) * (l - x) - e) * np.heaviside(x - (l / 2), 1)
        # print(get_B(i, init_lambds[i]))
        X = part1 + part2    
        X = X / np.linalg.norm(X)

        for _ in range(max_iter):
            X_wave = TMA(main_diag - init_lambds[i], upper_diag, lower_diag, X, x)
            X_wave = X_wave / np.linalg.norm(X_wave)

            lambd_new = np.dot(TMmulpt(main_diag, upper_diag, lower_diag, X_wave), X_wave) / np.dot(X_wave, X_wave)

            if abs(init_lambds[i] - lambd_new) < e:
                break

            init_lambds[i] = lambd_new
            X = X_wave

        
        num_lambds.append(init_lambds[i])
        num_func.append(X/np.linalg.norm(X))

    return num_lambds, num_func




init_lambds = [0.0355, 0.189, 0.39, 0.669, 1.13, 1.57]
num_lambds, num_func = rayleigh_method(main_diag, upper_diag, lower_diag, init_lambds, x)


In [36]:

for i in range(len(real_lambds)):
    print(f"real = {real_lambds[i]:.5f}, num = {num_lambds[i]:.5f}, delta = {(np.abs(real_lambds[i] - num_lambds[i])):.5f}")

real = 0.03651, num = 0.03651, delta = 0.00000
real = 0.19119, num = 0.19119, delta = 0.00000
real = 0.39478, num = 0.39478, delta = 0.00000
real = 0.67139, num = 0.67137, delta = 0.00001
real = 1.13545, num = 1.13541, delta = 0.00003
real = 1.57914, num = 1.57908, delta = 0.00005


In [37]:
real_functions = []
for i in range(len(real_lambds)):
    part1 = np.sin(np.sqrt(real_lambds[i]) * x) * np.heaviside((l / 2) - x, 0)
    part2 = get_B(i, real_lambds[i]) * np.sin(2 * np.sqrt(real_lambds[i]) * (l - x)) * np.heaviside(x - (l / 2), 1)
    real_functions.append((part1 + part2)/np.linalg.norm(part1 + part2))

In [48]:
fig = go.Figure()

sdvig = max(real_functions[0])
for i in range(len(real_lambds)):
    sdvig = max(max(real_functions[i]), sdvig)

sdvig *=2

for i in range(len(real_lambds)):
    fig.add_trace(go.Scatter(x=x, y = (real_functions[i]) + i * sdvig, name=f"λ={real_lambds[i]}", mode='lines'))
    
for i in range(len(num_lambds)):
    sgn = 1
    if real_functions[i][10] * num_func[i][10] < 0:
        sgn = -1
        
    fig.add_trace(go.Scatter(x=x, y= sgn * num_func[i] + i * sdvig, name=f"λ={num_lambds[i]}",
                             line=dict(color='black', dash='dash'),
                             mode='lines'))
fig.update_layout(
    title={'text': "С.ф. X(x) для своих λ",'x': 0.5,},
    xaxis_title='x',
    yaxis_title='X(x)'
)

fig.show()

In [39]:
fig = go.Figure()

for i in range(len(real_lambds)):
    sgn = 1
    if real_functions[i][10] * num_func[i][10] < 0:
        sgn = -1
    fig.add_trace(go.Scatter(x=x, y = abs(((real_functions[i] - sgn * num_func[i]))), name=f"λ={real_lambds[i]}", mode='lines'))


fig.update_layout(
    title={'text': "Абсолютная ошибка X(x) для своих λ",'x': 0.5,},
    xaxis_title='x',
    yaxis_title='X(x)',
    xaxis=dict(exponentformat='power'),  # Экспоненциальная запись для оси X
    yaxis=dict(exponentformat='power'),  # Экспоненциальная запись для оси Y
)

fig.show()

In [40]:
steps = [0.5, 0.1, 0.05, 0.01, 0.005]
ln_h = []
fault_array = []

for h in steps:
    x = np.linspace(0.0, l, int(l/h))
    n = len(x)
    p = 1 + 3 * np.heaviside(2*x - l, 0)
    
    main_diag, upper_diag, lower_diag = create_diagonals_L(n, h)

    num_lambds, num_func = rayleigh_method(main_diag, upper_diag, lower_diag, init_lambds, x)
    
    real_functions = []
    for i in range(len(real_lambds)):
        part1 = np.sin(np.sqrt(real_lambds[i]) * x) * np.heaviside((l / 2) - x, 0)
        part2 = get_B(i, real_lambds[i]) * np.sin(2 * np.sqrt(real_lambds[i]) * (l - x)) * np.heaviside(x - (l / 2), 1)
        real_functions.append((part1 + part2)/np.linalg.norm(part1 + part2))

    errors = []
    for i in range(len(num_lambds)):
        error = np.max(np.abs(real_functions[i] - num_func[i]))
        errors.append(error)
        
    fault_array.append(np.log(np.max(errors)))
    ln_h.append(np.log(h))

In [41]:

fig = go.Figure(data=[go.Scatter(x=ln_h, y=fault_array, mode='lines', line=dict(color='black'))])

fig.update_layout(
  title={'text': "Оценка точности метода",'x': 0.5,},
  xaxis_title='ln(h)',
  yaxis_title='ln(max_err)'
)

fig.show()